Merging and Cleaning Streaming History Data for Streaming History prediction

In [46]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pycaret.regression import *
import plotly

In [34]:
data=pd.DataFrame(pd.read_csv(r'./final_data/StreamingHistory/cleaned_date_wise.csv'))
data['date']=pd.to_datetime(data['date'].astype('datetime64[D]'))
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359 entries, 0 to 358
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           359 non-null    datetime64[ns]
 1   secondsPlayed  359 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 5.7 KB


In [35]:
data=data.set_index(data['date'],drop=False)
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 359 entries, 2020-02-11 to 2021-02-12
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           359 non-null    datetime64[ns]
 1   secondsPlayed  359 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 8.4 KB


In [4]:
reg=setup(data=data,target='secondsPlayed',train_size=0.75,n_jobs=-1)

,Description,Value
0,session_id,1599
1,Target,secondsPlayed
2,Original Data,"(359, 2)"
3,Missing Values,False
4,Numeric Features,0
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(269, 21)"


In [5]:
best = compare_models(fold=80)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
br,Bayesian Ridge,2572.6558,11541291.8529,3046.6888,-3.1718,0.9124,3.2045,0.0128
en,Elastic Net,2576.7249,11549038.3195,3054.1200,-3.4644,0.9138,3.1734,0.0115
ada,AdaBoost Regressor,2802.2738,13193056.4372,3245.6491,-3.9836,0.9520,3.5278,0.0270
omp,Orthogonal Matching Pursuit,2679.2261,12106578.4191,3154.8432,-4.5158,0.9397,3.3901,0.0100
llar,Lasso Least Angle Regression,2661.2071,12218122.3039,3173.3945,-5.7150,0.9310,2.8810,0.0124
lightgbm,Light Gradient Boosting Machine,2683.7535,12261532.5059,3192.7073,-5.7740,0.9358,3.0338,0.0279
ridge,Ridge Regression,2661.9896,12211264.6502,3173.3082,-5.8719,0.9308,2.8897,0.0107
lasso,Lasso Regression,2666.9802,12268705.7482,3181.8154,-5.9636,0.9319,2.8595,0.0124
lar,Least Angle Regression,2668.9301,12290227.0760,3184.8122,-6.0302,0.9321,2.8483,0.0129
huber,Huber Regressor,2638.0164,12360972.8895,3165.1508,-6.1932,0.9248,2.7186,0.0230


In [6]:
en=create_model(estimator='en',fold=150,round=4,verbose=False)

In [7]:
tuned_en=tune_model(en,fold=150,n_iter=200,optimize='MAE',verbose=False)

In [8]:
# bagged_br = ensemble_model(tuned_br, method = 'Bagging')
# train a adaboost classifier on dt with 100 estimators
further_tuned_en = ensemble_model(en,fold=100,round=4,n_estimators = 150,verbose=False)

In [9]:
evaluate_model(further_tuned_en)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [10]:
predicted_df= predict_model(further_tuned_en)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Elastic Net,2721.5291,12389279.0000,3519.8408,0.0121,1.1388,2.4368


In [12]:
finalized=finalize_model(further_tuned_en)
stream_history_model=save_model(finalized,'./model/streamhistory_model')

Transformation Pipeline and Model Succesfully Saved


Testing out with the model generated

In [13]:
history_predicted=load_model('./model/streamhistory_model')

Transformation Pipeline and Model Successfully Loaded


In [15]:
data={"Date":
            ["21-03-2021","22-05-2021"]
}
print(type(data))
data_test=pd.DataFrame(data)
print(type(data_test))
data_test['Date']=pd.to_datetime(data_test['Date'].astype('datetime64[D]'))
god=pd.DataFrame(data_test['Date'])
god.info()

<class 'dict'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    2 non-null      datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 144.0 bytes


In [23]:
god=pd.DataFrame(pd.read_csv('./final_data/StreamingHistory/date_wise_history.csv'))
prediction=predict_model(history_predicted,data=god)

Visualization

In [40]:
prediction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360 entries, 0 to 359
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           360 non-null    object 
 1   secondsPlayed  360 non-null    float64
 2   Label          360 non-null    float32
dtypes: float32(1), float64(1), object(1)
memory usage: 7.2+ KB


In [36]:
data_df=pd.DataFrame(pd.read_csv('./final_data/StreamingHistory/cleaned_date_wise.csv'))

In [37]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359 entries, 0 to 358
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           359 non-null    object 
 1   secondsPlayed  359 non-null    float64
dtypes: float64(1), object(1)
memory usage: 5.7+ KB


In [50]:
fig = px.line(prediction, x='date', y="secondsPlayed",title="Date Wise Data for 1 Year")
fig.show()

In [55]:
htmlfile=plotly.io.to_html(fig,full_html=False)
file=open("./plotly_output/date.html","w")
file.write(htmlfile)

3497104